In [1]:
import os
import shutil
from teemof.initialize import *
from teemof.run import initialize_trial

## Select sample files to modify

In [2]:
inp_lines, data_lines, qsub_lines = get_files(sample_files)

## Trial directory

In [3]:
trial_name = 'TrialX'

trial_dir = os.path.join(os.getcwd(), 'Trials', trial_name)
os.makedirs(trial_dir) if not os.path.exists(trial_dir) else None

## Single Trial

In [4]:
coefficient_list = [[1, 1, 0, 5], [2, 2, 0, 5], [1, 2, 0.1, 5]]
thermo_step = 100
masses = [[1, 39.948], [2, 20.1797]]

inp_lines = change_thermo(inp_lines, thermo=thermo_step)
inp_lines = change_pair_coeff(inp_lines, coefficient_list)
data_lines = change_masses(data_lines, masses)
qsub_lines = lammps_qsub(qsub_lines, name='TrialX', walltime='12:00:00', nodes=1, ppn=4, queue='shared')

number_of_runs = 10
initialize_trial(trial_dir, number_of_runs, inp_lines, data_lines, qsub_lines)

Initializing TrialX...
Generating seed numbers between 123456 and 123466
Run 1 / 10
Run 2 / 10
Run 3 / 10
Run 4 / 10
Run 5 / 10
Run 6 / 10
Run 7 / 10
Run 8 / 10
Run 9 / 10
Run 10 / 10


In [ ]:
epsilon = [0.01, 0.025, 0.05, 0.075, 0.1]

for trial_index, pc in enumerate(pair_coefficients):
    trial_dir = os.path.join(trials_dir, 'Trial3-%i' % (trial_index + 1))      # Initialize trial directory
    if not os.path.exists(trial_dir):
        os.mkdir(trial_dir)
        
    coefficient_list = [[1, 1, 0, 3.405],
                        [2, 2, 0, 3.405],
                        [1, 2, pc, 3.405]]
    
    with open(input_path, 'r') as sample_input:
        input_lines = sample_input.readlines()

    for i in range(5):
        run_dir = os.path.join(trial_dir, 'Run%i' % i)
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)

        input_lines = change_seed(input_lines, seed=seeds[i])
        input_lines = change_pair_coeff(input_lines, coefficient_list)
        job_name = 'PC%i-Run%i' % (pc, i)
        qsub_lines = lammps_qsub(qsub_path, name=job_name)

        inp = os.path.join(run_dir, 'in.cond')
        qs = os.path.join(run_dir, 'lammps_qsub.sh')
        dt = os.path.join(run_dir, 'lammps.data')

        export_lines(input_lines, inp)
        export_lines(qsub_lines, qs)
        shutil.copy(data_path, dt)